In [1]:
from bs4 import BeautifulSoup
import requests
import lxml
import os

### 1.1. Get the list of places

In [4]:
url2='https://www.atlasobscura.com/places?page='
sort='&sort=likes_count'

In [12]:
#DO NOT RUN THIS!
# create a file to put all the urls
f = open("urlpages.txt", "w")
for page in range(1, 401):
    url = url2 +str(page) +sort
    page_ = requests.get(url)
    soup = BeautifulSoup(page_.content, features='lxml')
    for a in soup.find_all('a', class_="content-card content-card-place"): #the class where the url is
        f.write(a.get('href')+'\n')

f.close()


### 1.2. Crawl places

In [7]:
def crawl():
    pathAncestor = os.path.join("./", "htmlpages") # Path
    os.mkdir(pathAncestor) # create the folder in the path
    
    f = open("urlpages2.txt", "w") #create txt file with urls
    count=1 #counting html files
    headpart='https://www.atlasobscura.com'
    
    for i in range(1, 401):
        os.makedirs(os.path.join(pathAncestor, 'page ' + str(i))) #create folder of html files
        
        url = url2 +str(i) +sort
        page_1 = requests.get(url)
        soup = BeautifulSoup(page_1.content, features='lxml')
        
        for a in soup.find_all('a', class_="content-card content-card-place"): #the class where the url is
            save=a.get('href')
            f.write(save +'\n')
            
            subdirectory = pathAncestor + "/page " + str(i) # select the corresponding folder to insert the html article
            article_name = "/article_"+str(count)+".html" 
            count+=1
            complete_path = subdirectory + article_name # insert the new complete path where create the html file
            
            with open(complete_path, "wb") as ip_file:
                link = headpart + save
                try:
                    page_2 = requests.get(link) # request the page
                except:
                    with open("failureRequest.txt", "a") as err_file: # if we loose the request place, we put into a file the link doesn't download well, then we set the "urlpages.txt" with these link
                        err_file.write(link)
                        err_file.close()

                soup = BeautifulSoup(page_2.text, features='lxml')

                ip_file.write(soup.encode('utf-8'))
                ip_file.close()

    f.close()

In [ ]:
crawl()

### 1.3 Parse downloaded pages


In [ ]:
def parse(tsv_writer, article): 
    
    with open(article, 'r', encoding="utf-8") as out_file: # for each html article downloaded scrape it!
        contents = out_file.read()
        soup = BeautifulSoup(contents, features="lxml") #parse the text
        
        try:
            placeName = soup.find('span', itemprop="ratingValue").text.strip()
        except:
            ratingValue = " "